## Mount Gdrive?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ****REMEMBER: change this to whatever your drive folder is
drive_folder = "/content/drive/My Drive/NLP CS395T/Artist-ic Endeavor"

## For Printing verses

In [ ]:
def print_verse(filename):
    artist_to_verses = get_artist_to_verses_model_output(filename)
    verse = artist_to_verses[1][37]
    text = " ".join(verse).strip()
    text = bpe_string_to_text(text)
    text = clean_lines(text)
    print(text)


In [ ]:
verses_file = 'verses_sentence.json'
print_verse('{}/{}'.format(drive_folder, verses_file))

i is young killer fake world you battle exploited produced of discussion every
i is young 21 niggas world rap lder shment ism dant my sonof
i is doggy real gangster aint analyze becoming lus ctive southern of ops christmas
i is young freaky the competition kansas stingin manipuprrra llyits fans with
i is diva my is dumb world bigg judged biothreatened those am entered
i is young eastside my real siren answer mbmaninovember its of dedicated
i is 21 my boss is dolls leads zihzih gniverits day plannin
i these young world my world mean is worked vals extinjesus aggression be
i is young savage my world is propheyeezy poetry us visual incredible about
i these boss other young bitch duplicate rapped lia ors egypt am hail they
i is di savage my world is darentilla identity its plannin ane
i is john is lil world ass barrio equjustice teaching 83 they do
i is thriller stupid same world one reaper psychogivjune fans they fact
i is trill same damn rap world mons haminiting what of gods
i is young 

## eval.py stuff

# ========== THIS IS CRUCIAL READ THIS ===========

ctrl-f ****REMEMBER


I left some comments of what to change when you are running stuff

In [3]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.util import ngrams
# from transformers import AutoTokenizer, AutoModelWithLMHead

import os
import json
import re
import subprocess
import locale
from tqdm import tqdm
from tqdm.autonotebook import trange
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd
%load_ext google.colab.data_table

# pip install transformers
# pip install pronouncing

# Self -bleu measures diversity between verse
# Distinct-n measures diversity in a verse

NUM_ARTISTS = 91
# ****REMEMBER: need to change this to the right location
RHYME_ANALYZER_JAR = '{}/RhymeApp.jar'.format(drive_folder)

def sel_bleu_artist(artist_corpus):
    '''
    Corpus is a list[list[str]], which is a list of verses.  Use to compute self-bleu
    '''
    total = 0
    list_of_references = []
    hypotheses = []
    for i in range(len(artist_corpus)):
        list_of_references.append(artist_corpus[:i] + artist_corpus[i+1:])
        hypotheses.append(artist_corpus[i])
    return corpus_bleu(list_of_references, hypotheses)

def sel_bleu_artist_avg(dataset, file_prefix):
    """
    dataset is a list[list[list[str]]], or a list of artist_corpus
    Returns per-artist self-bleu and dataset average
    """
    dataset_bleu = []
    for i in trange(len(dataset), desc="BLEU artist #"):
        artist = dataset[i]
        artist_bleu = sel_bleu_artist(artist)
        append_stat_to_txt_file("{}_sbleu".format(file_prefix), artist_bleu)
        dataset_bleu.append(artist_bleu)
    return dataset_bleu, sum(dataset_bleu) / len(dataset_bleu)


def distinct_n_verse(verse, n=1):
    """
    Calculates distinct n grams in a verse
    Verse should be list[str]
    """
    if len(verse) == 0:
        return 0.0  # Prevent a zero division
    distinct_ngrams = set(ngrams(verse, n))
    return len(distinct_ngrams) / len(verse)

def distinct_n_artist(artist_corpus, n=1):
    """
    Corpus is list[list[str]], or a list of verses
    Returns average distinct-n for an artist
    """

    verses_d = [distinct_n_verse(verse, n) for verse in artist_corpus]
    return sum(verses_d) / len(artist_corpus)

def distinct_n(corpus, n=1, file_prefix=''):
    """
    Corpus is list[list[list[str]]], or a list of artist-verses
    Returns list of artist distinct-n and average
    """
    artist_d = []
    for artist in corpus:
        artist_d_n = distinct_n_artist(artist, n)
        append_stat_to_txt_file("{}_d{}".format(file_prefix, n), artist_d_n)
        artist_d.append(artist_d_n)
    return artist_d, sum(artist_d) / len(corpus)

# def perplexity_artist(artist_corpus, tokenizer, model):
#     """
#     Returns the average perplexity of a artist-corpus using gpt2 as LM
#     """
#     ppls = []
#     for verse in artist_corpus:
#         tokenize_input = tokenizer.tokenize(verse)
#         tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
#         loss=model(tensor_input, lm_labels=tensor_input)
#         ppls.append(math.exp(loss))
#     return sum(ppls) / len(ppls)


def clean_lines(s):
    # Removes extra lines, and strips lines
    lines = [line.strip() for line in s.split('\n')]
    lines = list(filter(lambda s: s != '', lines))
    # reconstruct the lines together
    cleaned_lyrics = ''
    for line in lines:
        # concat lines together and add the end line token back
        cleaned_lyrics = cleaned_lyrics + line + '\n'
    return cleaned_lyrics

def calc_rhyme_density(bars):
    """
    bars: list of bpe tokens
    """
    text = " ".join(bars).strip()
    text = bpe_string_to_text(text)
    text = clean_lines(text)
    params = ['java', '-jar', RHYME_ANALYZER_JAR, text]
    output = subprocess.check_output(params)

    # convert to string
    encoding = locale.getdefaultlocale()[1]
    output = output.decode(encoding)

    result = {}
    for line in output.split('\n'):
        dv = line.split(':')

        if len(dv) == 2:
            key = dv[0].strip()
            value = float(dv[1].strip())
            result[key] = value

    statistics = result
    return statistics['Rhyme_Density'] if 'Rhyme_Density' in statistics else 0

# def dope_learning_rhyme_scores(bars):
#     text = " ".join(bars)
#     text = bpe_string_to_text(text)
#     l = Lyrics(text=text, language='en-us')
#     rl = l.get_avg_rhyme_length()
#     return rl

# def naive_rhyme_density(bars):
#     total_syllables = 0
#     rhymed_syllables = 0
#     words_used = set([word for bar in bars for word in bar.split()])
#     for bar in bars:
#         for word in bar.split():
#             p = pronouncing.phones_for_word(word)
#             if len(p) == 0:
#                 break
#             syllables = pronouncing.syllable_count(p[0])
#             total_syllables += syllables
#             has_rhyme = False
#             for rhyme in pronouncing.rhymes(word):
#                 if has_rhyme:
#                     break
#                 if rhyme in words_used:
#                     rhymed_syllables += syllables
#                     has_rhyme = True
#                     break
#     return rhymed_syllables/total_syllables


# TODO: Do artist similarity (cosine thing or crossentropy)

def rhyme_density(corpus, file_prefix):
    """
    Corpus is list[list[list[str]]], or a list of artist-verses
    Returns list of artist distinct-n and average
    """
    # print("\n===== COMPUTING RD =====\n")
    rds = []
    for i in trange(len(corpus)):
        artist = corpus[i]
        artist_rd_sum = 0
        for j in trange(len(artist), leave=False):
            artist_rd_sum = artist_rd_sum + calc_rhyme_density(artist[j])
        artist_rd = artist_rd_sum / len(artist)
        append_stat_to_txt_file("{}_rd".format(file_prefix), artist_rd)
        rds.append(artist_rd)
    return rds, sum(rds) / len(rds)

def calc_tfidf_score(gen_artist, vocab, dataset_artist):
    artist_dataset_verses_as_strings = [clean_lines(' '.join(verse)) for verse in dataset_artist]
    # combining all dataset lyrics to one string
    artist_dataset_all_verses = ' \n '.join(artist_dataset_verses_as_strings)

    all_verses_vectorizer = TfidfVectorizer(vocabulary = vocab)
    # vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
    all_verses_dataset_vectors = all_verses_vectorizer.fit_transform([artist_dataset_all_verses])

    verse_vectorizer = TfidfVectorizer(vocabulary = vocab)
    verse_vectors = verse_vectorizer.fit_transform(artist_dataset_verses_as_strings)

    avgs = []
    maxs = []
    for i in trange(len(gen_artist), desc='Verse # for Artist', leave=False):
        verse = gen_artist[i]
        cleaned_verse = [clean_lines(' '.join(verse))]

        gen_to_all_verses_vector = all_verses_vectorizer.transform(cleaned_verse)
        all_verses_sim = cosine_similarity(gen_to_all_verses_vector, all_verses_dataset_vectors).flatten()

        gen_to_each_verse_vector = verse_vectorizer.transform(cleaned_verse)
        verse_max_sim = cosine_similarity(gen_to_each_verse_vector, verse_vectors).flatten()

        avgs.append(np.mean(all_verses_sim))
        maxs.append(np.max(verse_max_sim))
    return np.mean(avgs), np.mean(maxs)

def get_tfidf_scores(corpus, file_prefix):
    artist_to_verses_dataset = get_artist_to_verses_dataset('{}/train.json'.format(drive_folder))
    with open('{}/bpe_string_token_to_int.json'.format(drive_folder)) as openfile:
        vocab = json.load(openfile).keys()
    average_tfidf_scores = []
    max_tfidf_scores = []
    for i in trange(len(corpus), desc='Artist #'):
        gen_artist = corpus[i]
        dataset_artist = artist_to_verses_dataset[i]
        avg_score, max_score = calc_tfidf_score(gen_artist, vocab, dataset_artist)
        append_stat_to_txt_file("{}_avg_sim_score".format(file_prefix), avg_score)
        append_stat_to_txt_file("{}_max_sim_score".format(file_prefix), max_score)
        average_tfidf_scores.append(avg_score)
        max_tfidf_scores.append(max_score)
    return average_tfidf_scores, max_tfidf_scores



def get_lyric_blocks(song, input_format):
    if input_format == "raw_song":
        return [song['lyrics']]
    elif input_format == "verses":
        return [verse['lyrics'] for verse in song['verses']]
    return []

# def get_artist_to_verses_marked_verses():
#     songs_dir = os.path.join("./", "data", "songs", "marked_verses")
#     songs_file = os.path.join(songs_dir, "_LIST")
#     song_list = read_list_from_file(songs_file)
#     artist_to_verses = {}
#     for song_name in song_list:
#         song_file = name_to_file_name(song_name)
#         with open('{}/{}.json'.format(songs_dir, song_file)) as jsonfile:
#             song = json.load(jsonfile)
#             for verse in song['verses']:
#                 artists = verse['artists'][0]
#                 lyrics = verse['lyrics']
#                 if verse['valid']:
#                     if not artists in artist_to_verses:
#                         artist_to_verses[artists] = []
#                     artist_to_verses[artists].append(lyrics)
#     print("Done reading things up")
#     return artist_to_verses

def bpe_string_to_text(s):
    return re.sub(r'(@@ )|(@@ ?$)', '', s)

def clean_tokens(s):
    s = re.sub(r' L | L', '\n', s)
    return re.sub(r'S ', '', s)

def remove_duplicate_new_lines(l):
    new_tokens = []
    for i in range(0, len(l)):
        curr_token = l[i]
        if curr_token != '\n':
            new_tokens.append(curr_token)
        elif i > 0:
            prev_token = l[i-1]
            if prev_token != '\n':
                new_tokens.append(curr_token)
    return new_tokens            
            

def replace_start_with_new_line_from_list(l):
    new_tokens = [token if token != 'S' else '\n' for token in l]
    return new_tokens

def replace_end_with_new_line_from_list(l):
    return [token if token != 'L' else '\n' for token in l]

def remove_end_tokens_from_list(l):
    return list(filter(lambda x: x != 'L', l))

def get_artist_to_verses_model_output(filename):
    with open(filename) as openfile:
        # this is in
        # {
            # 0 (artist id): [ (verses array)
            #     [(tokens array) a, b, c],
            # ]
        # }
        songs_json = json.load(openfile)
    artist_to_verses = []
    # instantiate list of lists, verses of artists
    for _ in range(0, NUM_ARTISTS):
        artist_to_verses.append([])
    for artist in songs_json:
        artist_index = int(artist) - 1
        # Remove the L and replace S with \n
        artist_to_verses[artist_index] = [remove_duplicate_new_lines(replace_start_with_new_line_from_list(replace_end_with_new_line_from_list(verse))) for verse in songs_json[artist]]
    return artist_to_verses

def get_artist_to_verses_dataset(filename):
    with open(filename) as openfile:
        # {artist_id: ,
        # lyrics: <string>}
        verses_json = json.load(openfile)
    artist_to_verses = []
    # instantiate list of lists, verses of artists
    for _ in range(0, NUM_ARTISTS):
        artist_to_verses.append([])
    for verse in verses_json:
        artist_index = int(verse['artist_id']) - 1
        lyrics_tokens = verse['lyrics'].split(' ')
        cleaned_tokens = replace_start_with_new_line_from_list(remove_end_tokens_from_list(lyrics_tokens))
        artist_to_verses[artist_index].append(cleaned_tokens)
    return artist_to_verses

def get_artist_to_verses_test():
  return [[['a', 'b', 'c'], ['a' , 'z', 'b']], ['hjkhkjhkjtdytr', 'asd']]

def append_stat_to_txt_file(filename, stat):
    # ****REMEMBER: make sure this is the write location
    with open("{}/{}.txt".format(drive_folder, filename), 'a') as openfile:
        openfile.write(str(stat))
        openfile.write('\n')

def write_out_stats_to_file(filename, stats_list):
    with open(filename, 'a') as openfile:
        json.dump(stats_list, openfile)

def artist_sim_matrix(start_artist, end_artist=91):
    file_prefix = "sim_matrix"
    artist_to_verses_dataset = get_artist_to_verses_dataset('{}/train.json'.format(drive_folder))
    with open('{}/bpe_string_token_to_int.json'.format(drive_folder)) as openfile:
        vocab = json.load(openfile).keys()
    average_tfidf_scores = []
    for i in trange(start_artist - 1, end_artist, desc='Artist #'):
        dataset_artist = artist_to_verses_dataset[i]
        for j in trange(len(artist_to_verses_dataset), desc='Artist comp #', leave=False):
            compare_artist = artist_to_verses_dataset[j]
            avg_score, _ = calc_tfidf_score(dataset_artist, vocab, compare_artist)
            append_stat_to_txt_file("{}_artist_{}".format(file_prefix, i + 1), avg_score)


def main(file_prefix, read_file_name):
    """
    the files will be of the form <file_prefix>_<metric>.txt
    """
    # ****REMEMBER: you might need to change this to the correct function to
    # format rhyme density, most likely will be something like
    # get_artist_to_verses_model_output(<some file name>)
    # you may have to write your own function if the format is different from
    # the assumed format in that function
    # ****REMEMBER: Make sure to start from the right artist
    artist_to_verses = get_artist_to_verses_model_output(read_file_name)
    per_artist_verses = artist_to_verses
    rd, avg_rd = rhyme_density(per_artist_verses, file_prefix)
    # ****REMEMBER: COMMENT THIS STUFF BELOW OUT IF YOU'RE ONLY DOING RHYME DENSITY
    s_bleu, s_bleu_avg = sel_bleu_artist_avg(per_artist_verses, file_prefix)
    distinct_1s, distinct_1_avg = distinct_n(per_artist_verses, 1, file_prefix)
    distinct_2s, distinct_2_avg = distinct_n(per_artist_verses, 2, file_prefix)
    distinct_3s, distinct_3_avg = distinct_n(per_artist_verses, 3, file_prefix)
    avg_sim_score, max_sim_score = get_tfidf_scores(per_artist_verses, file_prefix)
    return

def read_list_from_file(list_path):
    with open(list_path, encoding="utf-8") as listfile:
        l = listfile.readlines()
    return [i.strip() for i in l]

def list_to_floats(l):
    return [float(i) for i in l]

def summarize_metrics(file_prefix):
    # RD diff
    dataset_rds = list_to_floats(read_list_from_file('{}/train_dataset_rd.txt'.format(drive_folder)))
    model_rds = list_to_floats(read_list_from_file('{}/{}_rd.txt'.format(drive_folder, file_prefix)))
    rd_diff = mean_squared_error(dataset_rds, model_rds)
    s_bleu = list_to_floats(read_list_from_file('{}/{}_sbleu.txt'.format(drive_folder, file_prefix)))
    d1 = list_to_floats(read_list_from_file('{}/{}_d1.txt'.format(drive_folder, file_prefix)))
    d2 = list_to_floats(read_list_from_file('{}/{}_d2.txt'.format(drive_folder, file_prefix)))
    d3 = list_to_floats(read_list_from_file('{}/{}_d3.txt'.format(drive_folder, file_prefix)))
    avg_sim = list_to_floats(read_list_from_file('{}/{}_avg_sim_score.txt'.format(drive_folder, file_prefix)))
    max_sim = list_to_floats(read_list_from_file('{}/{}_max_sim_score.txt'.format(drive_folder, file_prefix)))
    with open("{}/{}_metrics_summary.txt".format(drive_folder, file_prefix), 'w') as openfile:
        openfile.write("RD Difference: MSE {} \n".format(str(rd_diff)))
        openfile.write("Self BLEU: Mean: {} Std: {}\n".format(str(np.mean(s_bleu)), str(np.std(s_bleu))))
        openfile.write("Distinct 1: Mean: {} Std: {}\n".format(str(np.mean(d1)), str(np.std(d1))))
        openfile.write("Distinct 2: Mean: {} Std: {}\n".format(str(np.mean(d2)), str(np.std(d2))))
        openfile.write("Distinct 3: Mean: {} Std: {}\n".format(str(np.mean(d3)), str(np.std(d3))))
        openfile.write("Avg Similarity: Mean: {} Std: {}\n".format(str(np.mean(avg_sim)), str(np.std(avg_sim))))
        openfile.write("Max Similarity: Mean: {} Std: {}\n".format(str(np.mean(max_sim)), str(np.std(max_sim))))

if __name__ == "__main__":
    # ****REMEMBER: change the file name if necessary
    pass


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
file_prefix = 'a9'
verses_file = 'metrics/bart_a9/bart_a9_all_verses.json'

main(file_prefix, '{}/{}'.format(drive_folder, verses_file))
summarize_metrics(file_prefix)

# Artist Sim matrix stuff 

In [ ]:
artist_sim_matrix(1)

In [ ]:
sim_matrix = pd.DataFrame()

In [ ]:
for i in range(0,91):
    artist_id = i + 1
    artist_scores_list = read_list_from_file("{}/sim_matrix_artist_{}.txt".format(drive_folder, i + 1))
    artist_scores = {j: float(artist_scores_list[j]) for j in range(0, len(artist_scores_list))}
    sim_matrix = sim_matrix.append(artist_scores, ignore_index=True)
sim_matrix.to_csv('{}/artist_sim_matrix.csv'.format(drive_folder))

In [25]:
sim_matrix = pd.read_csv("{}/metrics/artist_sim_analysis/artist_sim_matrix.csv".format(drive_folder))
artist_names = ['21 savage', '6ix9ine', 'dr dre', 'earl sweatshirt', 'ice cube', 'kayne west', 'kendrick lamar', 'kid cudi', 'pusha t', 'tyler the creator']
artist_indices = [1,4,22,25,35,43,45,46,66,85]
artist_indices2 = [i - 1 for i in artist_indices]
selected_matrix = sim_matrix.iloc[artist_indices, artist_indices]
selected_matrix

,0,3,21,24,34,42,44,45,65,84
1,0.400779,0.397974,0.368458,0.392804,0.309404,0.398335,0.348526,0.385975,0.348542,0.370582
4,0.384402,0.391507,0.367955,0.378612,0.307614,0.393165,0.346452,0.377578,0.348594,0.377512
22,0.483432,0.494475,0.484338,0.507726,0.452854,0.467712,0.474005,0.497557,0.464640,0.462303
25,0.506049,0.505546,0.488926,0.532019,0.485213,0.495101,0.490810,0.518072,0.469627,0.481991
35,0.465275,0.482832,0.475834,0.490084,0.437635,0.447798,0.464406,0.484374,0.454824,0.436268
43,0.393052,0.403957,0.397155,0.407879,0.356341,0.390999,0.388255,0.410177,0.387561,0.379128
45,0.458464,0.468076,0.452917,0.473001,0.411357,0.454393,0.447063,0.502293,0.439033,0.430547
46,0.358137,0.366130,0.356000,0.371136,0.320044,0.362458,0.349519,0.372804,0.347677,0.350191
66,0.481646,0.487098,0.472401,0.499298,0.460407,0.468562,0.472672,0.498208,0.455557,0.459408
85,0.412386,0.414200,0.402003,0.427230,0.365375,0.411195,0.388456,0.425418,0.387306,0.379820


In [28]:
row_rename_dict = {artist_indices[i] : artist_names[i] for i in range(len(artist_names))}
column_rename_dict2 = {str(artist_indices2[i]) : artist_names[i] for i in range(len(artist_names))}
final_selected_matrix = selected_matrix.rename(index=row_rename_dict, columns=column_rename_dict2)
final_selected_matrix = final_selected_matrix.applymap(lambda x: round(x, 3))
final_selected_matrix

,21 savage,6ix9ine,dr dre,earl sweatshirt,ice cube,kayne west,kendrick lamar,kid cudi,pusha t,tyler the creator
21 savage,0.401,0.398,0.368,0.393,0.309,0.398,0.349,0.386,0.349,0.371
6ix9ine,0.384,0.392,0.368,0.379,0.308,0.393,0.346,0.378,0.349,0.378
dr dre,0.483,0.494,0.484,0.508,0.453,0.468,0.474,0.498,0.465,0.462
earl sweatshirt,0.506,0.506,0.489,0.532,0.485,0.495,0.491,0.518,0.470,0.482
ice cube,0.465,0.483,0.476,0.490,0.438,0.448,0.464,0.484,0.455,0.436
kayne west,0.393,0.404,0.397,0.408,0.356,0.391,0.388,0.410,0.388,0.379
kendrick lamar,0.458,0.468,0.453,0.473,0.411,0.454,0.447,0.502,0.439,0.431
kid cudi,0.358,0.366,0.356,0.371,0.320,0.362,0.350,0.373,0.348,0.350
pusha t,0.482,0.487,0.472,0.499,0.460,0.469,0.473,0.498,0.456,0.459
tyler the creator,0.412,0.414,0.402,0.427,0.365,0.411,0.388,0.425,0.387,0.380


In [37]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

colored_matrix = final_selected_matrix.style.apply(highlight_max, axis = 1)
colored_matrix

,21 savage,6ix9ine,dr dre,earl sweatshirt,ice cube,kayne west,kendrick lamar,kid cudi,pusha t,tyler the creator
21 savage,0.401000,0.398000,0.368000,0.393000,0.309000,0.398000,0.349000,0.386000,0.349000,0.371000
6ix9ine,0.384000,0.392000,0.368000,0.379000,0.308000,0.393000,0.346000,0.378000,0.349000,0.378000
dr dre,0.483000,0.494000,0.484000,0.508000,0.453000,0.468000,0.474000,0.498000,0.465000,0.462000
earl sweatshirt,0.506000,0.506000,0.489000,0.532000,0.485000,0.495000,0.491000,0.518000,0.470000,0.482000
ice cube,0.465000,0.483000,0.476000,0.490000,0.438000,0.448000,0.464000,0.484000,0.455000,0.436000
kayne west,0.393000,0.404000,0.397000,0.408000,0.356000,0.391000,0.388000,0.410000,0.388000,0.379000
kendrick lamar,0.458000,0.468000,0.453000,0.473000,0.411000,0.454000,0.447000,0.502000,0.439000,0.431000
kid cudi,0.358000,0.366000,0.356000,0.371000,0.320000,0.362000,0.350000,0.373000,0.348000,0.350000
pusha t,0.482000,0.487000,0.472000,0.499000,0.460000,0.469000,0.473000,0.498000,0.456000,0.459000
tyler the creator,0.412000,0.414000,0.402000,0.427000,0.365000,0.411000,0.388000,0.425000,0.387000,0.380000


# Testing stuff

In [ ]:
l = ['a', 'asd', 'afsd', 'fad']
list(filter(lambda x: x== 'a', l))

['a']

In [ ]:
import codecs
f = codecs.open('testfile', 'w', 'utf8')
f.write("test")
f.close()

In [ ]:
from tqdm.autonotebook import trange
from time import sleep

for i in trange(5, leave=False):
    for j in trange(10, leave=False):
        sleep(.2)


In [ ]:
with open("{}/testing_stuff.txt".format(drive_folder), "a") as openfile:
    openfile.write('test')

In [ ]:
l = [1,2,3,4]
l[1:]

[2, 3, 4]

In [ ]:
# bad bart
# test = ['S', 'S', 'if', 'i', 'ever', 'got', 'caught', 'smokin', 'dope', 'it', 'must', 'be', 'adderall', 'L', 'S', 'S', 'you', 'gotta', 'trust', 'one', 'nigga', 'man', 'it', 'must', 'be', 'L', 'S', 'S', 'i', 'know', 'some', 'niggas', 'that', 'been', 'selling', 'dope', 'they', 'gon', 'sell', 'you', 'the', 'pot', 'L', 'S', 'S', 'they', 'brought', 'the', 'pistols', 'to', 'the', 'block', 'that', 'was', 'a', 'bastard', 'shot', 'L', 'S', 'S', 'im', 'a', 'slaughter', 'gang', 'with', 'shooters', 'gettin', 'chased', 'by', 'a', 'cop', 'L', 'S', 'S', 'nigga', 'act', 'like', 'he', 'dont', 'like', 'it', 'we', 'gon', 'run', 'him', 'over', 'with', 'the', 'glock', 'L', 'S', 'S', 'and', 'all', 'the', 'real', 'niggas', 'do', 'is', 'stay', 'humble', 'cause', 'they', 'dont', 'want', 'no', 'drama', 'L', 'S', 'S', 'a', 'few', 'bullets', 'will', 'cut', 'a', 'nigga', 'in', 'half', 'he', 'gon', 'need', 'a', 'morgue', 'L', 'S', 'S', 'i', 'got', 'my', 'pistols', 'dont', 'be', 'nice', 'we', 'use', 'a', 'body', 'bag', 'L', 'S', 'S', 'so', 'its', 'no', 'lacking', 'blood', 'on', 'the', 'dash', 'aint', 'no', 'blood', 'stains', 'nigga', 'L', 'S', 'S', 'slaughter', 'gang', 'freddie', 'kruger', 'playing', '21', 'games', 'nigga', 'L', 'S', 'S', '2', 'piece', 'chinchilla', 'L', 'S', 'S', 'or', 'you', 'could', 'call', 'me', 'frost', 'i', 'been', 'pullin', 'on', 'lame', 'niggas', 'L', 'S', 'S', 'i', 'been', 'pullin', 'on', 'bitches', 'i', 'been', 'kissin', 'on', 'some', 'hoes', 'L', 'S', 'S', 'all', 'you', 'do', 'is', 'trap', 'niggas', 'hope', 'the', 'cops', 'catch', 'us', 'L', 'S', 'S', 'bitch', 'i', 'can', 'feel', 'the', 'whoop', 'when', 'i', 'pulled', 'up', 'L']

# cvae
# test = ["S", "bia", "heeee@@", "thing", "rising", "thow", "dents", "in", "ita", "kross", "ent", "iously", "ave", "middle", "michigan", "S", "af", "dream", "lookin", "hoar@@", "gains", "with", "biological", "o@@", "canvas", "tw", "muti@@", "os", "oblock", "station", "S", "scrip@@", "nutty", "look", "livin", "nasir", "with", "ac", "nets", "u@@", "morals", "bmm", "brighter", "hours", "bulls", "S", "code@@", "breecky", "look", "belligerent", "meh", "get", "avenue", "in", "gra", "sting", "philosophers", "power", "graphic", "85", "S", "spiteful", "type", "ily", "way", "sitions", "ladder", "when", "stra@@", "vie", "form", "ing@@", "mack@@", "the", "rival", "S", "jailed", "2", "carib@@", "convicts", "shooting", "virus", "at", "the", "extre@@", "hospitable", "holidays", "isms", "trees", "wrld", "S", "arose", "on", "chico", "obligation", "nyquil", "rydin", "worm", "from", "makeshift", "ored", "muny", "bass", "apollo", "mid@@", "S", "extremes", "5", "way", "tuxedo", "flashing", "magnums", "when", "blicky", "at", "ills", "changes", "highness", "ctic", "february", "S", "pari@@", "2", "meds", "summers", "dictator", "back", "ability", "with", "gos", "fran@@", "abyss", "tin", "geo@@", "vera", "S", "hookup", "tink", "way", "faking", "tweaking", "with", "cage", "L", "S", "oped", "2", "ured", "squadron", "x3", "55", "with", "board", "accu@@", "ambi@@", "mem", "68", "country", "hydro", "S", "grandmaster", "on", "okay", "entertainment", "lakes", "lala", "with", "simons", "priest", "lega@@", "to", "housing", "terms", "the", "S", "sah", "musk", "back", "forgot", "beehive", "on", "crisis", "twi@@", "tipper", "ine", "kman", "trees", "fight", "designing", "S", "hustlin", "mush", "type", "milo", "depends", "do@@", "u@@", "assa@@", "bab@@", "befriend", "apartments", "checks", "evi@@", "light@@", "S", "fun@@", "on", "boss", "scrapes", "75", "impalas", "and", "lung", "to", "madman", "carolina", "yonkers", "13", "dit", "S", "dian", "balls", "thats", "mmmmm", "bending", "in", "deliver@@", "tigger", "farewell", "put", "century", "shaolin", "on@@", "skyline"]
# bart
# test= ['S', 'gang', 'parties', 'every', 'single', 'day', 'i', '<unk>', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'stra@@', '<unk>', '<unk>', 'woah', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'soul', '<unk>', '<unk>', 'for', '<unk>', '<unk>', 'you', '<unk>', '<unk>', 'believe', '<unk>', '<unk>', 'that', '<unk>', '<unk>', 'im', 'tryna', 'get', 'a', 'shot', 'at', '<unk>', '<unk>', 'der@@', '<unk>', '<unk>', 'doubt', '<unk>', 'S', 'mark', 'my', 'finger', '<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'all', 'on', '<unk>', '<unk>', 'right', '<unk>', '<unk>', 'act', 'on', 'me', 'and', 'make', 'me', '<unk>', '<unk>', 'heard', '<unk>', '<unk>', 'my', '<unk>', '<unk>', 'police', 'come', '<unk>', '<unk>', 'its', 'like', 'a', 'friend', 'of', 'mine', 'duckin', '<unk>', '<unk>', 'never', 'catch', '<unk>', '<unk>', 'doin', '<unk>', 'S', 'you', 'a', '<unk>', '<unk>', '<unk>', 'nigga', 'and', 'all', 'that', 'static', 'i', '<unk>', '<unk>', 'when', 'the', 'shit', '<unk>', '<unk>', 'five', '<unk>', '<unk>', 'it', '<unk>', '<unk>', 'my', '<unk>', '<unk>', 'lo@@', '<unk>', '<unk>', 'you', '<unk>', '<unk>', 'im', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'for', '<unk>', '<unk>', 'shit', '<unk>', 'for', 'my', 'niggas', '<unk>', '<unk>', 'S', 'when', 'im', 'locked', '<unk>', '<unk>', '<unk>', 'that', 'ass', 'down', 'thats', 'a', 'lock', '<unk>', '<unk>', 'im', '<unk>', '<unk>', 'right', '<unk>', '<unk>', 'sack', 'come', 'out', 'and', 'catch', 'a', 'fuck', 'nigga', 'when', 'he', 'come', 'home', '<unk>', '<unk>', 'crime', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'it', '<unk>', '<unk>', 'dont', '<unk>', '<unk>', 'on', 'shit', '<unk>', 'S', 'i', 'throw', 'it', 'by', 'the', '<unk>', '<unk>', '<unk>', 'u@@', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'mi@@', '<unk>', '<unk>', 'the', '<unk>', 'right', 'and', 'throw', 'it', 'up', 'like', 'a', 'motherfucking', '<unk>', '<unk>', 'dont', 'know', 'what', 'this', 'bitch', '<unk>', '<unk>', 're@@', '<unk>', '<unk>', 'religion', '<unk>', '<unk>', 'ver@@', '<unk>', '<unk>', 'feel', '<unk>', '<unk>', 'mines', 'S', 'and', 'im', 'from', 'the', 'dirty', '<unk>', '<unk>', '<unk>', 'a@@', '<unk>', '<unk>', 'like', 'im', 'from', '<unk>', '<unk>', 'already', '<unk>', '<unk>', 'my', '<unk>', '<unk>', 'yeah', 'yeah', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'right', '<unk>', '<unk>', 'dont', '<unk>', '<unk>', 'all', 'my', 'niggas', 'get', '<unk>', '<unk>', 'sleep', 'like', 'big', '<unk>', '<unk>', 'that', '<unk>', '<unk>', 'S', 'im', 'in', 'the', '<unk>', '<unk>', '<unk>', 'the', '<unk>', 'the', 'young', '<unk>', '<unk>', 'good', 'niggas', '<unk>', '<unk>', 'oh', 'doing', 'business', '<unk>', '<unk>', 'nigga', 'oh', '<unk>', '<unk>', 'rhyme', '<unk>', '<unk>', 'shit', 'we', 'gettin', 'money', 'shit', 'gettin', 'and', 'gettin', 'money', 'everyday', 'over', 'here', 'getting', 'money', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'head', 'getting', 'S', 'rock', 'a', '40', 'on', 'me', 'cause', 'im', 'on', '<unk>', '<unk>', '<unk>', 'br@@', '<unk>', '<unk>', 'gra@@', '<unk>', '<unk>', 'my', 'nigga', 'brand', 'new', '<unk>', '<unk>', 'wife', 'and', '<unk>', '<unk>', 'just', 'came', 'home', 'from', 'feelin', 'a', 'lil', '<unk>', '<unk>', 'right', 'from', 'doing', '<unk>', '<unk>', 'swa@@', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'oh', 'aint', 'S', 'oh', 'aint', 'no', 'mo', '<unk>', '<unk>', '<unk>', 'nigga', 'we', 'o', '<unk>', '<unk>', 'crime', '<unk>', '<unk>', 'four', 'my', 'nigga', 'we', '<unk>', '<unk>', 'ta@@', '<unk>', '<unk>', 'ea@@', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'yall', 'you', 'say', 'we', 'got', 'the', 'finest', 'bitches', 'in', 'the', '<unk>', 'tu@@', '<unk>', '<unk>', 'dont', 'leave', 'that', 'shit', 'cause', 'S', 'when', 'im', 'riding', 'off', 'in', 'the', '6', 'my', 'niggas', '<unk>', '<unk>', '<unk>', 'wont', 'wake', '<unk>', '<unk>', 'law', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'loud', 'the', '<unk>', 'wont', '<unk>', '<unk>', 'show', 'bitch', 'make', 'the', 'po', '<unk>', '<unk>', 'for', '<unk>', '<unk>', 'yall', 'my', '<unk>', '<unk>', 'blow', 'gon', 'post', '<unk>', 'S', 'oh', 'she', 'love', 'that', 'boy', 'ayy', 'she', 'gon', 'spend', 'it', 'on', '<unk>', '<unk>', '<unk>', 'shit', 'oh', 'she', 'miss', 'my', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'wont', '<unk>', '<unk>', 'huh', '<unk>', '<unk>', 'your', '<unk>', '<unk>', 'thats', '<unk>', '<unk>', 'when', 'i', 'run', 'shit', 'but', 'i', 'still', 'wont', 'leave', 'the', 'house', 'and', '<unk>', 'S', 'when', 'i', 'seen', 'a', 'ak', 'in', 'the', 'house', '<unk>', '<unk>', '<unk>', 'clothes', '<unk>', '<unk>', 'say', 'money', 'make', 'me', 'dance', '<unk>', '<unk>', 'that', '<unk>', '<unk>', 'my', '<unk>', '<unk>', 'o@@', '<unk>', '<unk>', 'all', '<unk>', '<unk>', 'your', '<unk>', '<unk>', 'i', '<unk>', '<unk>', 'try', 'and', 'do', 'like', 'me', 'just', 'like', '<unk>', '<unk>', 'it', 'S', 'run', 'up', 'on', 'you', 'run', 'up', '<unk>', '<unk>', '<unk>', '<mask>', '<unk>', '<unk>', 'yeah', 'wet', 'it', '<unk>', '<unk>', 'your', '<unk>', '<unk>', 'you', '<unk>', '<unk>', 'on@@', '<unk>', '<unk>', 'that', '<unk>', '<unk>', 'on', '<unk>', '<unk>', 'say', 'you', 'wanna', 'fuck', 'a', 'lil', '<unk>', '<unk>', 'right', 'fuck', 'lil', 'nigga', 'you', '<unk>', 'nigga', '<unk>', '<unk>', 'S', 'bitch', 'im', '<unk>', '<unk>', '<unk>', 'its', 'right', '<unk>', '<unk>', 'yall', '<unk>', '<unk>', 'blow', '<unk>', '<unk>', 'ction@@', '<unk>', '<unk>', 'a@@', '<unk>', '<unk>', 'your', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'ohh', '<unk>', '<unk>', 'for', '<unk>', '<unk>', 're@@', '<unk>', '<unk>', 'yeah', '<unk>', '<unk>', 'crime', 'dont', 'touch', 'my', '<unk>', '<unk>', 'my', '<unk>', 'crime', '<unk>', 'S', '<unk>', '<unk>', '<unk>', 'check', 'lets', 'get', 'to', 'this', '<unk>', '<unk>', 'hand', '<unk>', '<unk>', 'show', '<unk>', '<unk>', 'me', '<unk>', '<unk>', 'crime', '<unk>', '<unk>', 'r@@', '<unk>', '<unk>', 'for', 'my', '<unk>', '<unk>', 'my', '<unk>', 'h@@', '<unk>', '<unk>', 'hope', 'that', 'these', 'hoes', 'got', '<unk>', '<unk>', 'it', '<unk>', '<unk>', 'got', '<unk>', 'my', 'bible', 'written', 'S', 'yeah', '<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'crime', 'for', 'your', 'boy', 'and', 'your', '<unk>', '<unk>', 'bra@@', '<unk>', '<unk>', 'you', '<unk>', '<unk>', 'justice', 'for', 'your', '<unk>', 'crime', '<unk>', '<unk>', 'it', '<unk>', '<unk>', 'your', '<unk>', 'e@@', '<unk>', '<unk>', 'for', 'your', 'bitch', 'for', 'making', 'me', '<unk>', '<unk>', 'war', '<unk>', '<unk>', 'my', '<unk>']

test= ['S', 'in', 'a', 'bimmer', 'hustling', 'like', 'its', 'a', 'cage', 'L', 'S', 'goin', 'in', 'my', 'bimmer', 'now', 'its', 'outer', 'space', 'L', 'S', 'that', 'bitch', 'funny', 'and', 'i', 'promise', 'i', 'aint', 'talkin', 'bout', 'cake', 'i', 'dont', 'want', 'no', 'cake', 'L', 'S', 'tell', 'these', 'hoes', 'sit', 'back', 'and', 'laugh', 'while', 'i', 'show', 'her', 'how', 'to', 'get', 'it', 'yeah', 'L', 'S', 'and', 'they', 'actin', 'like', 'they', 'got', 'no', 'soul', 'got', 'so', 'many', 'cups', 'you', 'need', 'a', 'belly', 'tuck', 'or', 'some', 'ass@@', 'ets', 'L', 'S', 'pull', 'up', 'on', 'your', 'baby', 'mama', 'im', 'like', 'lil', 'kodak', 'why', 'you', 'say', 'that', 'L', 'S', 'got', 'my', 'chopper', 'from', 'my', 'mama', 'yeah', 'i', 'used', 'to', 'keep', 'it', 'silent', 'L', 'S', 'lil', 'jeezy', 'be', 'on', 'them', 'snitches', 'lil', 'nigga', 'he', 'be', 'on', 'some', 'go@@', 'ba@@', 'b', 'shit', 'L', 'S', 'they', 'tried', 'to', 'pay', 'me', 'for', 'my', 'jewels', 'i', 'aint', 'talkin', 'bout', 'no', 'hockey', 'shit', 'L', 'S', 'young', 'savage', 'dope', 'fiend', 'with', 'a', 'chopper', 'get', 'the', 'glock', 'and', 'a', 'ruger', 'bitch', 'L', 'S', 'i', 'aint', 'got', 'no', 'time', 'to', 'snitch', 'bitch', 'i', 'need', 'some', 'handles', 'L', 'S', 'ill', 'fuck', 'yo', 'sister', 'then', 'i', 'fuck', 'yo', 'thot', 'bitch', 'i', 'keep', 'goin', 'to', 'sleep', 'L', 'S', 'bitch', 'i', 'keep', 'on', 'goin', 'to', 'sleep', 'im', 'tryna', 'make', 'a', 'million', 'off', 'of', 'this', 'dope', 'L', 'S', 'how', 'you', 'got', 'me', 'pissed', 'off', 'i', 'got', 'the', 'shottie', 'bitch', 'im', 'cold', 'like', 'a', 'tough', 'talk', 'L', 'S', 'aye', 'bang', 'bang', 'L', 'S', 'ridin', 'with', 'my', 'toolie', 'on', 'my', 'hip', 'no', 'crease', 'L']



In [ ]:
gittest = remove_duplicate_new_lines(replace_start_with_new_line_from_list(replace_end_with_new_line_from_list(test)))
print(' '.join(test))

in a bimmer hustling like its a cage 
 goin in my bimmer now its outer space 
 that bitch funny and i promise i aint talkin bout cake i dont want no cake 
 tell these hoes sit back and laugh while i show her how to get it yeah 
 and they actin like they got no soul got so many cups you need a belly tuck or some ass@@ ets 
 pull up on your baby mama im like lil kodak why you say that 
 got my chopper from my mama yeah i used to keep it silent 
 lil jeezy be on them snitches lil nigga he be on some go@@ ba@@ b shit 
 they tried to pay me for my jewels i aint talkin bout no hockey shit 
 young savage dope fiend with a chopper get the glock and a ruger bitch 
 i aint got no time to snitch bitch i need some handles 
 ill fuck yo sister then i fuck yo thot bitch i keep goin to sleep 
 bitch i keep on goin to sleep im tryna make a million off of this dope 
 how you got me pissed off i got the shottie bitch im cold like a tough talk 
 aye bang bang 
 ridin with my toolie on my hip no crease 



In [ ]:
for i in trange(5, 10):
    print(i)

In [ ]:
append_stat_to_txt_file("12313testse", 1)